# Aula 6: SQL

## Introdução

- SQL é uma linguagem padrão para trabalhar com bancos de dados relacionais
- É uma linguagem declarativa e que não necessita de profundos conhecimentos de programação para que alguém possa começar a escrever queries
- SQL é acrônimo de Structured Query Language, literalmente a linguagem struturada para realizar queries.
- Apesar de existirem diversos bancos de dados relacionais, cada um com sua funcionalidade, todos possuem comandos básicos que podem ser aplicados em qualquer banco de dados:

![](https://cdn-images-1.medium.com/fit/t/1600/480/1*LcXr-gRpSfe63XjdCVa15A.png)

- Nessa aula usaremos o site https://www.db4free.net/, que utiliza MySQL
- Antes de começarmos, criem uma conta no DB4Free

## Códigos

É possível **Inserir** dados na tabela, por exemplo:

    INSERT INTO Usuarios(nome, idade, sexo, salario) VALUES ("Joaquim Silva",60,"M",200)
    
É possível **Atualizar** a tabela, por exemplo: 

    UPDATE Usuarios SET salario=2000 WHERE Nome="Joaquim Silva"
    
É possível **Remover** os dados, por exemplo:

    DELETE FROM Usuarios WHERE Nome="Joaquim Silva"
    
É Possível **Consultar** a tabela, por exemplo :

## Conexao no db4free

- Para trabalhar com bancos de dados, temos que fazer uma conexão do nosso jupyter notebook com o banco de dados

In [1]:
# importando libs necessárias
import mysql.connector

In [17]:
#Dados do banco de dados db4free
user = 'ossamum'
password = 'letscode'
host = 'db4free.net'
database = 'ossamum_db'
port = 3306

In [18]:
#Conectando
con = mysql.connector.connect(user=user, 
                              password=password,
                              host=host,
                              database=database, port=port)

cursor = con.cursor()

## Queries de criação e manipulação de tabelas

- Agora que estabelecemos a conexão com o banco de dados, podemos começar a aplicar as queries
- Podemos escrever as queries dentro da variável `query` e o `cursor.execute(query)` aplica a query

### Show tables

Mostrar tabelas do banco de dados

In [36]:
query = '''
    SHOW TABLES
'''

cursor.execute(query)

Buscar o resultado da query com o método `fetchall`

In [37]:
cursor.fetchall()

[(bytearray(b'empresas2'),)]

### Create table

Criar tabela

In [29]:
query = '''CREATE TABLE empresas(estabelecimento CHAR(40),
                               receita INT,
                               custo INT,
                               anomes INT)'''
cursor.execute(query)

### Drop table

Deletar tabela

In [23]:
query = '''
    DROP TABLE empresas
'''

cursor.execute(query)

###  Insert into table

Agora que criamos uma tabela, podemos inserir valores nela

In [30]:
val = ("MCDonalds","5000","4000","201912")

query = f'''INSERT INTO empresas(estabelecimento,receita, custo, anomes) 
VALUES ('{val[0]}','{val[1]}','{val[2]}','{val[3]}')'''


cursor.execute(query)

###  Select

Podemos checar os valores com o comando `Select`

In [31]:
query = "SELECT * FROM empresas"

cursor.execute(query)

resultado = cursor.fetchall()


print(resultado)

[('MCDonalds', 5000, 4000, 201912)]


- No caso de consulta, podemos usar a biblioteca de manipulação da dados `pandas`

In [7]:
import pandas as pd

In [33]:
pd.read_sql(query, con)

,estabelecimento,receita,custo,anomes
0,MCDonalds,5000,4000,201912


###  Inserindo mais dados

In [34]:
val =  [("Wendys","6000","4500","202001"),
        ("Patties","900","900","202001"),
        ('Zebeleo', '5', '10000', '202002'),
        ('Koburger', '3000', '2500', '202002')]


sql = '''INSERT INTO empresas(estabelecimento, receita, custo, anomes) 
          VALUES (%s,%s,%s,%s)'''

cursor.executemany(sql, val)

Podemos criar uma função no python para inserir dados diretamente no banco de dados

In [37]:
def input_data(con=con, cursor=cursor):
    print('Nome da Empresa :')
    nome = input()
    print('Receita da Empresa :')
    receita = input()
    print('Custo da Empresa :')
    custo = input()
    print('Anomes :')
    anomes = input()
    
    
    query = f'''INSERT INTO empresas(estabelecimento,receita, custo, anomes) 
                    VALUES ('{nome}','{receita}','{custo}','{anomes}')'''
    
    cursor.execute(query)
    
    print('Data inserted!')

In [38]:
input_data()

Nome da Empresa :
Bullguer
Receita da Empresa :
400
Custo da Empresa :
600
Anomes :
202103
Data inserted!


### Delete 

Podemos deletar uma linha do banco de dados

In [39]:
query = "DELETE FROM empresas Where estabelecimento = 'Bullguer'"
cursor.execute(query)

### Update

Podemos atualizar um valor da tabela

In [42]:
query = "UPDATE empresas set estabelecimento = 'Mequi_1000' Where estabelecimento = 'MCDonalds'"
cursor.execute(query)

## Queries de consulta

In [42]:
import sqlalchemy
engine = sqlalchemy.create_engine(f'mysql://{user}:{password}@{host}:{port}/{database}') # connect to server

### SELECT e FROM

- Já vimos o `SELECT` e `FROM` acima
    - `SELECT` seleciona as colunas que queremos trazer
        - `*` seleciona toda das colunas
    - `FROM` seleciona a tabela que analisaremos
- Para facilitar a consulta, podemos aplicar o `LIMIT`, que define o número de linhas que a query irá trazer

In [46]:
query = '''
SELECT *
FROM AtrasoCartaoCredito
LIMIT 5
'''

pd.read_sql_query(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,0,1,0,3,homem,22,1,7250.0,517.857143,604.166667,MG,Morada Nova de Minas,Região Sudeste,54.924242
1,1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
2,2,3,1,3,mulher,26,0,7925.0,566.071429,660.416667,PR,Agudos do Sul,Região Sul,50.801282
3,3,4,1,1,mulher,35,1,531.0,37.928571,44.250000,PR,Congonhinhas,Região Sul,2.528571
4,4,5,0,3,homem,35,0,805.0,57.500000,67.083333,BA,Morpará,Região Nordeste,3.833333


### WHERE

- Podemos filtar algumas observações com o comando `WHERE`

In [47]:
query = '''
SELECT *
FROM AtrasoCartaoCredito
WHERE Sexo = 'homem'
LIMIT 5
'''

pd.read_sql_query(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,0,1,0,3,homem,22,1,7250.0,517.857143,604.166667,MG,Morada Nova de Minas,Região Sudeste,54.924242
1,4,5,0,3,homem,35,0,805.0,57.500000,67.083333,BA,Morpará,Região Nordeste,3.833333
2,5,6,0,3,homem,22,0,84583.0,6041.642857,7048.583333,AL,Carneiros,Região Nordeste,640.780303
3,6,7,0,1,homem,54,0,51862.0,3704.428571,4321.833333,PR,Figueira,Região Sul,160.067901
4,7,8,0,3,homem,20,3,21075.0,1505.357143,1756.250000,BA,Ouriçangas,Região Nordeste,175625.000000


- `<>` significa "diferente", ou seja, selecionaremos todos `purchase_id` menos `0`

In [48]:
query = '''
SELECT *
FROM AtrasoCartaoCredito
WHERE Sexo <> 'homem'
LIMIT 5
'''

pd.read_sql_query(query, engine)

,index,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
1,2,3,1,3,mulher,26,0,7925.0,566.071429,660.416667,PR,Agudos do Sul,Região Sul,50.801282
2,3,4,1,1,mulher,35,1,531.0,37.928571,44.250000,PR,Congonhinhas,Região Sul,2.528571
3,8,9,1,3,mulher,27,0,11133.0,795.214286,927.750000,MG,Comercinho,Região Sudeste,68.722222
4,9,10,1,2,mulher,14,1,3007.0,214.785714,250.583333,MG,Japonvar,Região Sudeste,35.797619


### DISTINCT

- Podemos analisar os valores únicos de uma coluna

In [50]:
query = '''
SELECT DISTINCT GrupoEconomico
FROM AtrasoCartaoCredito

'''

pd.read_sql_query(query, engine)

,GrupoEconomico
0,3
1,1
2,2


### GROUP BY

- Podemos agrupar a tabela por uma chave e sumarizar os valores 
- Por exemplo, podemos analisar o número de cada `product_category` 
- `GROUP BY` sempre está acompanhando de alguma função de agregação: `COUNT()`, `SUM()`, `AVG()`

In [51]:
query = '''

SELECT GrupoEconomico, COUNT(*)
FROM AtrasoCartaoCredito 
GROUP BY GrupoEconomico

'''

pd.read_sql_query(query, engine)

,GrupoEconomico,COUNT(*)
0,3,491
1,1,216
2,2,184


### ORDER BY

Podemos ordernar a consulta por alguma coluna com `ORDER BY` em ordem crescente

In [52]:
query = '''

SELECT GrupoEconomico, COUNT(*)
FROM AtrasoCartaoCredito 
GROUP BY GrupoEconomico
ORDER BY GrupoEconomico

'''

pd.read_sql_query(query, engine)

,GrupoEconomico,COUNT(*)
0,1,216
1,2,184
2,3,491


- Ou em ordem descrescente
- Também podemos renomear a coluna com o alias `AS`

In [53]:
query = '''

SELECT GrupoEconomico, COUNT(*) AS num_GrupoEconomico
FROM AtrasoCartaoCredito 
GROUP BY GrupoEconomico
ORDER BY GrupoEconomico DESC

'''

pd.read_sql_query(query, engine)

,GrupoEconomico,num_GrupoEconomico
0,3,491
1,2,184
2,1,216


### HAVING

Quando queremos aplicar um filtro após fazer o `GROUP BY`, usamos o comando `HAVING`

In [54]:
query = '''

SELECT GrupoEconomico, COUNT(*) AS num_GrupoEconomico
FROM AtrasoCartaoCredito 
GROUP BY GrupoEconomico
HAVING COUNT(*) > 200

'''

pd.read_sql_query(query, engine)

,GrupoEconomico,num_GrupoEconomico
0,3,491
1,1,216
